##Section 2.4. A Partitioned Matrix View of the Simplex Method


In this section we see the basic command for creating partitioned matrices (or block matrices) in Numpy.  We then find the matrices used in the text to give a partitioned matrix version of the simplex algorithm.

In [0]:
import numpy as np
import pandas as pd

We create partitioned matrices using <code>np.block</code> in Numpy.  To create the matrices used in the introduction in the text, we use the following.

In [0]:
A=np.array([[1,3],
            [2,0]])
B=np.array([[2,1],
            [-1,1]])
C=np.array([[-5,0],
            [2,6]])
I2=np.eye(2)
M=np.block([[A,B],
            [C,I2]])
print(M)

[[ 1.  3.  2.  1.]
 [ 2.  0. -1.  1.]
 [-5.  0.  1.  0.]
 [ 2.  6.  0.  1.]]


In Numpy's blocks, one dimensional arrays work as rows.  We sometimes want to convert these to columns, which can be done with the following technique.

In [0]:
np.zeros(3)[:,np.newaxis]

---
Now we follow the text in using partitioned matrices for the simplex algorithm.  The next codeblock needs to be executed to load our row reducing fuctions.  The following codeblock contains the data for the FuelPro LP and can be edited for different problems.  The third codeblock needs to be executed to set up the tableaux.

In [0]:
def SwapRow(arrayA,rowi,rowj): #interchange rows i and j
    arrayB=arrayA.copy()
    arrayB[rowi]=arrayA[rowj]
    arrayB[rowj]=arrayA[rowi]
    return arrayB
def AddRow(arrayA,rowi,rowj,s): #add s * row j to row i
    arrayB=arrayA.copy()
    arrayB=arrayB.astype(float) #to avoid division errors with integer arrays
    arrayB[rowi]=arrayA[rowi]+s*arrayA[rowj]
    return arrayB
def MultiplyRow(arrayA,rowi,s): #multiply row i by s
    arrayB=arrayA.copy()
    arrayB=arrayB.astype(float) #to avoid division errors with integer arrays
    arrayB[rowi]=s*arrayA[rowi]
    return arrayB
def Pivot(arrayA,rowi,colj): #pivot matrix at row i and column j
    arrayB=arrayA.copy()
    arrayB=MultiplyRow(arrayA,rowi,1/arrayA[rowi,colj])
    for x in range(0,rowi):
        arrayB=AddRow(arrayB,x,rowi,-arrayB[x,colj])
    for x in range(rowi+1,arrayB.shape[0]):
        arrayB=AddRow(arrayB,x,rowi,-arrayB[x,colj])
    return arrayB

In [0]:
c = np.array([4,3])
#create a one-dimensional array of objective function coefficients
A = np.array([[1,0],
             [2,2],
             [3,2]])
#create a two-dimensional array of constraint coefficients
b = np.array([8,28,32])
#create a one-dimensional array of constraint bounds

In [0]:
n=len(c)
m=len(b)
#find the number of decision variables and constraints, respectively
x=['x_'+str(i+1) for i in range(n)]
s=['s_'+str(i+1) for i in range(m)]
Labels=['z']+x+s+['RHS']
#create labels for decision and slack variables
def Tableau(M):
    return pd.DataFrame(M,columns=Labels)
#procedure to add column labels to an LPMatrix
LPMatrix = np.block([
    [1,-c,np.zeros(m+1)],
    [np.zeros((m,1)),A,np.identity(m),b[:,np.newaxis]]
])
#create array corresponding to the initial tableau
def RowRatios(M,c):
    for i in range(m):
        if (M[i+1,c]>0.001):
            print("Row", i+1, "Ratio =", M[i+1,-1]/M[i+1,c])
        else:
            print("Row", i+1, "Ratio Undefined")
#row ratio test for LPMatrix M and column c
def Iterate(M,r,c):
    M2=MultiplyRow(M,r,1/M[r,c])
    M2=Pivot(M2,r,c)
    return M2

In [0]:
TableauMatrix1=LPMatrix
Tableau(TableauMatrix1)

,z,x_1,x_2,s_1,s_2,RHS
0,1.0,-10.0,-16.0,0.0,0.0,0.0
1,0.0,2.0,2.0,1.0,0.0,10.0
2,0.0,1.0,2.0,0.0,1.0,8.0


Now we can define the block matrix used for the first iteration of the simplex algorithm, assuming that we determine what this matrix should be by hand.

In [0]:
y1=np.array([4,0,0])
M1=np.array([[1,0,0],
            [-2,1,0],
            [-3,0,1]])
S1=np.block([[1,y],
          [np.zeros(3)[:,np.newaxis],M]])
print(S1)

Now we can multiply to carry out the first iteration of the simplex algorithm.

In [0]:
TableauMatrix2=S1@TableauMatrix1
Tableau(TableauMatrix2)

,z,x_1,x_2,s_1,s_2,RHS
0,1.0,-2.0,0.0,0.0,8.0,64.0
1,0.0,1.0,0.0,1.0,-1.0,2.0
2,0.0,0.5,1.0,0.0,0.5,4.0


Alternatively, it is possible to compute $y_1$ and $M_1$ by making the pivot in an appropriate submatrix of the initial tableau.  We take the column that we pivot in, together with columns that give an identity matrix.  Performing the pivot gives a partitioned matrix containing $y_1$ and $M_1$.

In [0]:
MatA=TableauMatrix1[:,[2,3,4]]
print(MatA)
MatB=Pivot(MatA,2,0)
print(MatB)

[[-16.   0.   0.]
 [  2.   1.   0.]
 [  2.   0.   1.]]
[[ 0.   0.   8. ]
 [ 0.   1.  -1. ]
 [ 1.   0.   0.5]]


In [0]:
y1=MatB[0,1:]
print(y1)
M1=MatB[1:,1:]
print(M1)

[0. 8.]
[[ 1.  -1. ]
 [ 0.   0.5]]


In [0]:
S1=np.block([[1,y1],
          [np.zeros(2)[:,np.newaxis],M1]])

Next we compute the matrices $y_2$ and $M_2$ used for the second iteration of the simplex algorithm.

In [0]:
Tableau(TableauMatrix2)

,z,x_1,x_2,s_1,s_2,RHS
0,1.0,-2.0,0.0,0.0,8.0,64.0
1,0.0,1.0,0.0,1.0,-1.0,2.0
2,0.0,0.5,1.0,0.0,0.5,4.0


In [0]:
RowRatios(TableauMatrix2,1)

Row 1 Ratio = 2.0
Row 2 Ratio = 8.0


In [0]:
MatA=TableauMatrix2[:,[1,3,2]]
print(MatA)
MatB=Pivot(MatA,1,0)
print(MatB)

[[-2.   0.   0. ]
 [ 1.   1.   0. ]
 [ 0.5  0.   1. ]]
[[ 0.   2.   0. ]
 [ 1.   1.   0. ]
 [ 0.  -0.5  1. ]]


In [0]:
y2=MatB[0,1:]
print(y2)
M2=MatB[1:,1:]
print(M2)
S2=np.block([[1,y2],
             [np.zeros(2)[:,np.newaxis],M2]])
print(S2)

[2. 0.]
[[ 1.   0. ]
 [-0.5  1. ]]
[[ 1.   2.   0. ]
 [ 0.   1.   0. ]
 [ 0.  -0.5  1. ]]


Next we carry out the second iteration of the simplex algorithm.

In [0]:
TableauMatrix3=S2@TableauMatrix2
Tableau(TableauMatrix3)

For Waypoint 2.4.3, we should verify the tableau matrix formula using $y_1$, $y_2$, $M_1$, and $M_2$.  The first calculation is shown here and the rest are left as an exercise.

In [0]:
-c+(y1+y2@M1)@A

array([0., 0.])

Finally we compute the matrices $y_3$ and $M_3$ for the third iteration of the simplex algorithm.

In [0]:
Tableau(TableauMatrix3)

In [0]:
RowRatios(TableauMatrix3,3)

In [0]:
MatA=TableauMatrix3[:,[3,1,4,2]]
print(MatA)
MatB=Pivot(MatA,2,0)
print(MatB)

In [0]:
y3=MatB[0,1:]
print(y3)
M3=MatB[1:,1:]
print(M3)
S3=np.block([[1,y3],
             [np.zeros(3)[:,np.newaxis],M3]])
print(S3)

In [0]:
TableauMatrix4=S3@TableauMatrix3
Tableau(TableauMatrix4)

---
We can also confirm the calculations of Theorem 2.4.1 in this case.

In [0]:
M=M3@M2@M1
print(M)

In [0]:
y=y1+y2@M1+y3@M2@M1
print(y)

In [0]:
-c+y@A

In [0]:
y@b

In [0]:
M@A

In [0]:
M@b[:,np.newaxis]

In [0]:
S = np.block([[1,y],
              [np.zeros(3)[:,np.newaxis],M]])
print(S)
T = np.block([[1,-c,np.zeros(3),0],
              [np.zeros(3)[:,np.newaxis],A,np.eye(3),b[:,np.newaxis]]])
print(T)
print(S@T)

In [0]:
print(np.block([[1,-c+y@A,y,y@b],
                [np.zeros(3)[:,np.newaxis],M@A,M,M@b[:,np.newaxis]]])
     )